In [ ]:
!pip install git+https://github.com/salaniz/pycocoevalcap.git

In [6]:


import os
import pandas as pd
import json
import string
from pycocoevalcap.cider.cider import Cider

# ===== Paths =====
folder_path = "/content/drive/MyDrive/phi3mini"
output_json_folder = os.path.join(folder_path, "cider_dicts")
os.makedirs(output_json_folder, exist_ok=True)

baseline_file = "mscoco_captions.csv"
model_files = {
    "Flux-Dev": "fluxdev_meta.csv",
    "SDXL": "sdxl_meta.csv",
    "SD2": "sd2_meta.csv"
}

# ===== Text cleaning function =====
def clean_text(text):
    if pd.isna(text) or str(text).strip() == "":
        return None
    text = str(text).lower().strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# ===== Load baseline =====
baseline_df = pd.read_csv(os.path.join(folder_path, baseline_file))
baseline_df["mscoco_caption"] = baseline_df["mscoco_caption"].apply(clean_text)

# ===== Convert CSV to CIDEr dictionaries =====
def create_cider_dicts(model_file):
    df = pd.read_csv(os.path.join(folder_path, model_file))
    df["Meta Caption"] = df["Meta Caption"].apply(clean_text)

    # Merge baseline and model by image_name
    merged = pd.merge(
        baseline_df, df, on="image_name", how="inner"
    )

    refs = {}
    cands = {}

    for _, row in merged.iterrows():
        image_id = str(row["image_name"])
        ref_text = row["mscoco_caption"]
        cand_text = row["Meta Caption"]
        if ref_text and cand_text:
            refs[image_id] = [ref_text]
            cands[image_id] = [cand_text]

    return refs, cands

# ===== Process all models and save JSONs =====
for model_name, model_file in model_files.items():
    refs, cands = create_cider_dicts(model_file)

    refs_json_path = os.path.join(output_json_folder, f"{model_name}_refs.json")
    cands_json_path = os.path.join(output_json_folder, f"{model_name}_cands.json")

    with open(refs_json_path, "w") as f:
        json.dump(refs, f, indent=4)
    with open(cands_json_path, "w") as f:
        json.dump(cands, f, indent=4)

    print(f"Saved CIDEr dictionaries for {model_name}:")
    print(f"  refs -> {refs_json_path}")
    print(f"  cands -> {cands_json_path}")

# ===== Compute CIDEr scores =====
average_scores = {}
for model_name in model_files.keys():
    refs_path = os.path.join(output_json_folder, f"{model_name}_refs.json")
    cands_path = os.path.join(output_json_folder, f"{model_name}_cands.json")

    with open(refs_path) as f:
        refs = json.load(f)
    with open(cands_path) as f:
        cands = json.load(f)

    try:
        scorer = Cider()
        score, _ = scorer.compute_score(refs, cands)
        average_scores[model_name] = score
        print(f"CIDEr score for {model_name}: {score:.4f}")
    except Exception as e:
        print(f"Error computing CIDEr for {model_name}: {e}")
        average_scores[model_name] = None

# ===== Save average scores =====
output_json = os.path.join(folder_path, "cider_average_scores.json")
with open(output_json, "w") as f:
    json.dump(average_scores, f, indent=4)

print("Average CIDEr scores saved to:", output_json)
print(average_scores)


Saved CIDEr dictionaries for Flux-Dev:
  refs -> /content/drive/MyDrive/phi3mini/cider_dicts/Flux-Dev_refs.json
  cands -> /content/drive/MyDrive/phi3mini/cider_dicts/Flux-Dev_cands.json
Saved CIDEr dictionaries for SDXL:
  refs -> /content/drive/MyDrive/phi3mini/cider_dicts/SDXL_refs.json
  cands -> /content/drive/MyDrive/phi3mini/cider_dicts/SDXL_cands.json
Saved CIDEr dictionaries for SD2:
  refs -> /content/drive/MyDrive/phi3mini/cider_dicts/SD2_refs.json
  cands -> /content/drive/MyDrive/phi3mini/cider_dicts/SD2_cands.json
CIDEr score for Flux-Dev: 0.0000
CIDEr score for SDXL: 0.0000
CIDEr score for SD2: 0.0026
Average CIDEr scores saved to: /content/drive/MyDrive/phi3mini/cider_average_scores.json
{'Flux-Dev': np.float64(4.846772725301889e-05), 'SDXL': np.float64(8.937773580845634e-06), 'SD2': np.float64(0.0025563240724885993)}
